### 將 Google-NNLM-ZH 以 Keras Embedding Layer 的形式讀取
+ 從[這個網頁](https://tfhub.dev/google/nnlm-zh-dim128-with-normalization/2)下載模型檔案
+ 使用 `tensorflow_hub` 套件讀取存在本地端的模型
+ 該資料夾要包含 `.pb` 檔
+ 檔案資料夾結構：
    ```
    .
    ├── NNLM-ZH
    │   ├── assets
    │   │   └── tokens.txt
    │   ├── saved_model.pb
    │   └── variables
    │       ├── variables.data-00000-of-00001
    │       └── variables.index
    └── NNLM-ZH-Demo.ipynb
    ```

In [1]:
import tensorflow_hub as hub

emb_layer = hub.KerasLayer('./NNLM-ZH')

### 讀取純文字字典檔
+ 新鮮的 `./assets/tokens.txt` 裡面基本上都是簡體中文
+ 此例字典檔內有 968,075 個詞

In [2]:
tokens = open('./NNLM-ZH/assets/tokens.txt', 'r', encoding='UTF-8').read().strip().split('\n')
print(f'Total tokens: {len(tokens)}')

Total tokens: 968075


### 探索 Embedding Layer
+ 檢視 `emb_layer.get_weights()` 可以發現只有一組權重
+ 檢視 `weights[0].shape` 發現大小為 971177 x 128
  + **!! 大小與字典檔並不一致 !!**

In [3]:
weights = emb_layer.get_weights()
print(f'Number of weights: {len(weights)}')
print(f'Shape of the weights: {weights[0].shape}')

Number of weights: 1
Shape of the weights: (971177, 128)


### 基本使用方法
+ 在 TF2.0 裡面可以直接使用 Layer 呼叫來獲得 Text Embedding

In [4]:
result = emb_layer(['今天天氣真好'])
result.shape

TensorShape([1, 128])

### 測試單詞
1. 使用 Layer Call
2. 使用 Token Index 定位 Layer Weight

In [5]:
token = '猫'

# Layer Call
t1 = emb_layer([token])

# Token Index
idx = tokens.index(token)
t2 = emb_layer.get_weights()[0][idx]

# Compare
(t1 == t2).numpy().all()

True

### 大小寫敏感測試
+ Runa 有出現在字典裡
+ 結果顯示是 Case-Sensitive

In [6]:
t1 = emb_layer(['runa'])
idx = tokens.index('Runa')
t2 = emb_layer.get_weights()[0][idx]
(t1 == t2).numpy().all()

False

### OOV 測試
+ runa 與貓並未出現在字典檔內

In [7]:
t1 = emb_layer(['runa'])
idx = tokens.index('<UNK>')
t2 = emb_layer.get_weights()[0][idx]
(t1 == t2).numpy().all()

False

In [8]:
t1 = emb_layer(['貓'])
idx = tokens.index('<UNK>')
t2 = emb_layer.get_weights()[0][idx]
(t1 == t2).numpy().all()

False

### 從字典檔內隨機挑選字詞測試
+ 隨機檢視的結果發現有些 Tokens 裡面有 `##` 這種與 BERT Tokenizer 相似的處理符號
+ 甚至可能比 BERT Tokenizer 更複雜

In [9]:
import random

for i in range(5):
    token = random.choice(tokens)
    print(f'Test Token: {token}')
    token_id = tokens.index(token)
    print(f'Token ID: {token_id}')

    t1 = emb_layer([token])
    t2 = emb_layer.get_weights()[0][token_id]
    print(f'Pass Test: {(t1 == t2).numpy().all()}\n')

Test Token: 蔡学胜
Token ID: 335996
Pass Test: True

Test Token: PiPO
Token ID: 378640
Pass Test: True

Test Token: 陆建德
Token ID: 243462
Pass Test: True

Test Token: 煤耗
Token ID: 53311
Pass Test: True

Test Token: 雷长林
Token ID: 327671
Pass Test: True



In [10]:
token = '###.##a/b/g/n'
print(f'Test Token: {token}')
token_id = tokens.index(token)
print(f'Token ID: {token_id}')

t1 = emb_layer([token])
t2 = emb_layer.get_weights()[0][token_id]
print(f'Pass Test: {(t1 == t2).numpy().all()}\n')

Test Token: ###.##a/b/g/n
Token ID: 99382
Pass Test: False



### Sentence Embedding Re-implement
+ 參考 TF-Hub 的介紹頁面使用 `tf.nn.embedding_lookup_sparse` 與 sqrtn combiner 重現 Google NNLM 的 Sentence Embedding 製作方法

In [11]:
import tensorflow as tf

emb_weights = emb_layer.get_weights()[0]

def to_ids(sent):
    segs = sent.split(' ')
    ids = [list(map(tokens.index, segs))]
    return ids

def to_sparse(sent):
    ids = to_ids(sent)
    ids = tf.convert_to_tensor(ids)
    return tf.sparse.from_dense(ids)

def op_weights(sent):    
    return tf.nn.embedding_lookup_sparse(emb_weights, to_sparse(sent), None, combiner='sqrtn')

def op_hand(sent):
    import numpy as np
    ids = to_ids(sent)
    arr = np.array([emb_weights[i] for i in ids]).sum(axis=1)
    arr /= ((len(ids) + 2) ** 0.5) # 此處尚不知為何要 +2

    return arr

def equal(t1, t2):
    return (t1 == t2).numpy().all()

In [12]:
sent = '猫 和 狗'

t1 = emb_layer([sent])
t2 = op_weights(sent)
t3 = op_hand(sent)

equal(t1, t2), equal(t2, t3), equal(t1, t3)

(True, True, True)

### 結論
+ 九成的 Google NNLM 操作已經被成功解析
+ 證實了字典檔與模型權重之間確實有關聯
+ 尚無法理解 UNK Token 的行為

In [13]:
t1 = emb_layer(['貓'])

emb_weights = emb_layer.get_weights()[0]
ids = to_sparse('<UNK>')
t2 = tf.nn.embedding_lookup_sparse(emb_weights, ids, None, combiner='sqrtn')
(t1 == t2).numpy().all()

False